In [114]:
import os
from langchain_community.llms import ollama
from langchain.prompts import FewShotPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [115]:
def retres(llm,temp,ref,tar,span,ser,cat):
    chain = temp | llm | StrOutputParser()    
    response = chain.invoke({"ref": ref, "tar": tar,"error_span":span,"error_category":cat,"severity":ser}) 
    return response

In [116]:
import pandas as pd
df=pd.read_csv('dataset.csv')
df=df[:20]

In [117]:
example=[]
for i in range(0,11):
    a={"ref":df['Ref'][i],"tar":df['Tar'][i],"error_span":df['error_span'][i],"severity":df['severity'][i],"error_category":df['error_cat'][i]}
    example.append(a)

In [118]:
example

[{'ref': 'A cat sat on a mat.',
  'tar': 'A mat sat on a cat.',
  'error_span': '"mat sat on a cat"',
  'severity': 'major',
  'error_category': 'Unnatural'},
 {'ref': 'The mother sang a lullaby to the sleepy cat',
  'tar': 'The chair sang a lullaby to the sleepy cat',
  'error_span': '"chair"',
  'severity': 'major',
  'error_category': 'Unnatural'},
 {'ref': 'Break a leg',
  'tar': 'Break one of your legs',
  'error_span': '"one of your legs"',
  'severity': 'major',
  'error_category': 'Mistranslation, Addition'},
 {'ref': 'Please make yourself at home',
  'tar': 'Please create your home',
  'error_span': '"create your home"',
  'severity': 'major',
  'error_category': 'Mistranslation'},
 {'ref': 'The software is buggy',
  'tar': 'The software has insects',
  'error_span': '"software has insects"',
  'severity': 'major',
  'error_category': 'Mistranslation'},
 {'ref': 'Although it was raining, she went for a walk',
  'tar': 'Although it is raining, she went for a walk',
  'error_spa

In [119]:
template = """
Reference sentence: {ref}
Target sentence: {tar}
Error Span: {error_span}
Error Category {error_category}
\
"""

temp = PromptTemplate(
    input_variables=["ref", "tar","error_span","severity","error_category"],
    template=template,
)
fewshotprompt=FewShotPromptTemplate(
    examples=example,
    example_prompt=temp,
    input_variables=["ref", "tar","error_span","severity","error_category"],
    suffix="Reference Sentence:{ref}\nTarget Sentence:{tar}\nError Span:{error_span}\nError Category {error_category}",
    prefix="Identify the error span, error category and severity (in terms of minor, major) in the target sentence with respect to the given reference sentence. No need of any explanation or corrected sentence. Error category and error span can be one or more in the given sentences. Generate error category error span and severity in json format."
)



## LLAMA2

In [120]:
for i in range(0,len(df)):
    print('Sentence '+str(i)+'\n')
    print('Reference sentence '+df['Ref'][i]+'\n')
    print('Target sentence '+df['Tar'][i]+'\n')
    print(retres(ollama.Ollama(temperature=0,model='llama2'),fewshotprompt,df['Ref'][i],df['Tar'][i],df['error_span'][i],df['severity'][i],df['error_cat'][i]))
    print('\n\n')

Sentence 0

Reference sentence A cat sat on a mat.

Target sentence A mat sat on a cat.

Here are the errors in the target sentences with respect to the reference sentences, along with their error categories and severities:

1. Target sentence: A mat sat on a cat.
Error span: "mat sat on a cat"
Error category: Unnatural
Severity: Error
2. Target sentence: The chair sang a lullaby to the sleepy cat.
Error span: "chair"
Error category: Unnatural
Severity: Error
3. Target sentence: Break one of your legs.
Error span: "one of your legs"
Error category: Mistranslation, Addition
Severity: Major
4. Target sentence: Please create your home.
Error span: "create your home"
Error category: Mistranslation
Severity: Minor
5. Target sentence: The software has insects.
Error span: "software has insects"
Error category: Mistranslation
Severity: Major
6. Target sentence: Although it is raining, she went for a walk.
Error span: "it is raining, she went for a walk"
Error category: Grammatical error
Sever

Here are the errors in the target sentences along with their corresponding error categories and severity levels:

1. Target sentence: A mat sat on a cat.
Error category: Unnatural
Error span: "mat sat on a cat"
Severity level: Error (minor)
2. Target sentence: The chair sang a lullaby to the sleepy cat.
Error category: Unnatural
Error span: "chair"
Severity level: Error (major)
3. Target sentence: Break one of your legs.
Error category: Mistranslation, Addition
Error span: "one of your legs"
Severity level: Error (major)
4. Target sentence: Please create your home.
Error category: Mistranslation
Error span: "create your home"
Severity level: Error (minor)
5. Target sentence: The software has insects.
Error category: Mistranslation
Error span: "software has insects"
Severity level: Error (major)
6. Target sentence: Although it was raining, she went for a walk.
Error category: Grammatical error
Error span: "it is raining, she went for a walk"
Severity level: Error (minor)
7. Target sente

{
"reference_sentences": [
"A cat sat on a mat.",
"The mother sang a lullaby to the sleepy cat.",
"Break a leg.",
"Please make yourself at home.",
"The software is buggy.",
"Although it was raining, she went for a walk.",
"I found the buyer with the wrong order.",
"They have been studying French for three years.",
"She will arrive at the airport at noon tomorrow."
],
"target_sentences": [
"A mat sat on a cat.",
"The chair sang a lullaby to the sleepy cat.",
"Break one of your legs.",
"Please create your home.",
"The software has insects.",
"It is raining, she went for a walk.",
"I found the buyer of the wrong order before.",
"They have been studying French for three years and after.",
"She will arrive at the airport at noon."
],
"error_categories": [
{
"name": "Unnatural",
"span": "mat sat on a cat"
},
{
"name": "Mistranslation",
"span": "chair",
"addition": true
},
{
"name": "Mistranslation",
"span": "one of your legs",
"migration": true
},
{
"name": "Grammatical error",
"span": "it i

Here are the errors in the target sentences with respect to the reference sentences, along with their error categories and severities:

1. Target sentence: A mat sat on a cat.
Reference sentence: A cat sat on a mat.
Error span: "mat sat on a cat"
Error category: Unnatural
Severity: Error
2. Target sentence: The chair sang a lullaby to the sleepy cat.
Reference sentence: The mother sang a lullaby to the sleepy cat.
Error span: "chair"
Error category: Unnatural
Severity: Error
3. Target sentence: Break one of your legs.
Reference sentence: Break a leg.
Error span: "one of your legs"
Error category: Mistranslation, Addition
Severity: Major
4. Target sentence: Please create your home.
Reference sentence: Please make yourself at home.
Error span: "create your home"
Error category: Mistranslation
Severity: Minor
5. Target sentence: The software has insects.
Reference sentence: The software is buggy.
Error span: "software has insects"
Error category: Mistranslation
Severity: Major
6. Target s

Here are the errors in the target sentences with respect to the reference sentences:

1. Target sentence: A mat sat on a cat.
Error category: Unnatural
Error span: "mat sat on a cat"
2. Target sentence: The chair sang a lullaby to the sleepy cat.
Error category: Unnatural
Error span: "chair"
3. Target sentence: Break one of your legs.
Error category: Mistranslation, Addition
Error span: "one of your legs"
4. Target sentence: Please create your home.
Error category: Mistranslation
Error span: "create your home"
5. Target sentence: The software has insects.
Error category: Mistranslation
Error span: "software has insects"
6. Target sentence: Although it is raining, she went for a walk.
Error category: Grammatical error
Error span: "it is raining, she went for a walk"
7. Target sentence: I found the buyer of the wrong order before.
Error category: Grammatical error
Error span: "of the wrong order before"
8. Target sentence: They has been studying French for three years.
Error category: Gr

## LLAMA3

In [121]:
for i in range(0,len(df)):
    print('Sentence '+str(i)+'\n')
    print('Reference sentence '+df['Ref'][i]+'\n')
    print('Target sentence '+df['Tar'][i]+'\n')
    print(retres(ollama.Ollama(temperature=0,model='llama3'),fewshotprompt,df['Ref'][i],df['Tar'][i],df['error_span'][i],df['severity'][i],df['error_cat'][i]))
    print('\n\n')

Sentence 0

Reference sentence A cat sat on a mat.

Target sentence A mat sat on a cat.

Here are the errors in JSON format:

[
  {
    "error_span": "mat sat on a cat",
    "error_category": "Unnatural",
    "severity": "Major"
  },
  {
    "error_span": "chair",
    "error_category": "Unnatural",
    "severity": "Major"
  },
  {
    "error_span": "one of your legs",
    "error_category": "Mistranslation, Addition",
    "severity": "Major"
  },
  {
    "error_span": "create your home",
    "error_category": "Mistranslation",
    "severity": "Major"
  },
  {
    "error_span": "software has insects",
    "error_category": "Mistranslation",
    "severity": "Major"
  },
  {
    "error_span": "it is raining, she went for a walk",
    "error_category": "Grammatical error",
    "severity": "Major"
  },
  {
    "error_span": "of the wrong order before",
    "error_category": "Grammatical error",
    "severity": "Major"
  },
  {
    "error_span": "they has been",
    "error_category": "Grammat

Here are the errors in JSON format:

```
[
  {
    "error_span": "mat sat on a cat",
    "error_category": "Unnatural",
    "severity": "Major"
  },
  {
    "error_span": "chair",
    "error_category": "Unnatural",
    "severity": "Major"
  },
  {
    "error_span": "one of your legs",
    "error_category": "Mistranslation, Addition",
    "severity": "Major"
  },
  {
    "error_span": "create your home",
    "error_category": "Mistranslation",
    "severity": "Minor"
  },
  {
    "error_span": "software has insects",
    "error_category": "Mistranslation",
    "severity": "Major"
  },
  {
    "error_span": "it is raining, she went for a walk",
    "error_category": "Grammatical error",
    "severity": "Major"
  },
  {
    "error_span": "of the wrong order before",
    "error_category": "Grammatical error",
    "severity": "Major"
  },
  {
    "error_span": "they has been",
    "error_category": "Grammatical error",
    "severity": "Major"
  },
  {
    "error_span": "tomorrow",
    "erro

Here are the errors in JSON format:

```
[
  {
    "error_span": "mat sat on a cat",
    "error_category": "Unnatural",
    "severity": "Major"
  },
  {
    "error_span": "chair",
    "error_category": "Unnatural",
    "severity": "Major"
  },
  {
    "error_span": "one of your legs",
    "error_category": "Mistranslation, Addition",
    "severity": "Major"
  },
  {
    "error_span": "create your home",
    "error_category": "Mistranslation",
    "severity": "Major"
  },
  {
    "error_span": "software has insects",
    "error_category": "Mistranslation",
    "severity": "Major"
  },
  {
    "error_span": "it is raining, she went for a walk",
    "error_category": "Grammatical error",
    "severity": "Major"
  },
  {
    "error_span": "of the wrong order before",
    "error_category": "Grammatical error",
    "severity": "Major"
  },
  {
    "error_span": "they has been",
    "error_category": "Grammatical error",
    "severity": "Major"
  },
  {
    "error_span": "tomorrow",
    "erro

Here are the errors in JSON format:

```
[
  {
    "error_span": "mat sat on a cat",
    "error_category": "Unnatural",
    "severity": "Major"
  },
  {
    "error_span": "chair",
    "error_category": "Unnatural",
    "severity": "Major"
  },
  {
    "error_span": "one of your legs",
    "error_category": "Mistranslation, Addition",
    "severity": "Major"
  },
  {
    "error_span": "create your home",
    "error_category": "Mistranslation",
    "severity": "Major"
  },
  {
    "error_span": "software has insects",
    "error_category": "Mistranslation",
    "severity": "Major"
  },
  {
    "error_span": "it is raining, she went for a walk",
    "error_category": "Grammatical error",
    "severity": "Major"
  },
  {
    "error_span": "of the wrong order before",
    "error_category": "Grammatical error",
    "severity": "Major"
  },
  {
    "error_span": "they has been",
    "error_category": "Grammatical error",
    "severity": "Major"
  },
  {
    "error_span": "tomorrow",
    "erro

## Mistral

In [122]:
for i in range(0,len(df)):
    print('Sentence '+str(i)+'\n')
    print('Reference sentence '+df['Ref'][i]+'\n')
    print('Target sentence '+df['Tar'][i]+'\n')
    print(retres(ollama.Ollama(temperature=0,model='mistral'),fewshotprompt,df['Ref'][i],df['Tar'][i],df['error_span'][i],df['severity'][i],df['error_cat'][i]))
    print('\n\n')

Sentence 0

Reference sentence A cat sat on a mat.

Target sentence A mat sat on a cat.

 {
  "Error Span": ["mat sat on a cat"],
  "Error Category": "Unnatural",
  "Severity": "Major"
}



Sentence 1

Reference sentence The mother sang a lullaby to the sleepy cat

Target sentence The chair sang a lullaby to the sleepy cat

 {
  "Error Span": ["chair"],
  "Error Category": "Unnatural"
}



Sentence 2

Reference sentence Break a leg

Target sentence Break one of your legs

 {
  "Error Span": ["A mat sat on a cat", "create your home", "software has insects", "it is raining, she went for a walk", "of the wrong order before", "they has been", "tomorrow", "and after the schedule", "waked up"],
  "Error Category": ["Unnatural", "Mistranslation", "Mistranslation", "Grammatical error", "Grammatical error", "Grammatical error", "Ommission", "Addition, Ommission", "Spelling & Typological"]
}



Sentence 3

Reference sentence Please make yourself at home

Target sentence Please create your home



 {
  "Error Span": ["less than"],
  "Error Category": "Ommission"
}



Sentence 16

Reference sentence It has been 1 hour and it still has not been delivered yet.

Target sentence It's been an hour, and it hasn't arrived yet

 {
  "Error Span": ["cat", "chair", "one of your legs", "create your home", "software has insects", "it is raining, she went for a walk", "of the wrong order before", "they has been", "tomorrow", "and after the schedule", "waked up"],
  "Error Category": ["Unnatural", "Unnatural", "Mistranslation", "Mistranslation", "Mistranslation", "Grammatical error", "Grammatical error", "Grammatical error", "Ommission", "Addition, Ommission", "Spelling & Typological"],
  "Severity": ["Minor", "Minor", "Major", "Major", "Major", "Minor", "Minor", "Major", "Minor", "Minor", "Minor"]
}



Sentence 17

Reference sentence Then I will use my points to re-place the order, thank you.

Target sentence Then I'll re order with points. Thank you

 {
  "Error Span": ["mat sat on a cat", "

## TinyDolphin

In [123]:
for i in range(0,len(df)):
    print('Sentence '+str(i)+'\n')
    print('Reference sentence '+df['Ref'][i]+'\n')
    print('Target sentence '+df['Tar'][i]+'\n')
    print(retres(ollama.Ollama(temperature=0,model='tinydolphin'),fewshotprompt,df['Ref'][i],df['Tar'][i],df['error_span'][i],df['severity'][i],df['error_cat'][i]))
    print('\n\n')

Sentence 0

Reference sentence A cat sat on a mat.

Target sentence A mat sat on a cat.

 {
  "error_span": ["cat", "mat"],
  "error_category": "mismatched_word",
  "severity": "minor"
}<|im_end|>



Sentence 1

Reference sentence The mother sang a lullaby to the sleepy cat

Target sentence The chair sang a lullaby to the sleepy cat

 The sentence should be "the chair sang a lullaby to the sleepy cat."

In this case, the error in the sentence is that it uses "chair" as the subject of the sentence. This is incorrect because "chair" is an object and not a subject. Therefore, the correct sentence would be:

"The chair sang a lullaby to the sleepy cat."<|im_end|>



Sentence 2

Reference sentence Break a leg

Target sentence Break one of your legs

 Break a leg to you! This is an error in the sentence where "legs" is added to the phrase "one of your legs." The correct translation should be "break one of your legs," as it refers to a specific part of the body.<|im_end|>



Sentence 3

Refer

 {
  "errorSpan": ["mat sat on a cat", "chair sang a lullaby to the sleepy cat"],
  "errorCategory": ["Unnatural", "Mistranslation", "Grammatical error", "Addition", "Ommision"],
  "severity": {
    "Unnatural": 1,
    "Mistranslation": 2,
    "Grammatical error": 3,
    "Addition": 4,
    "Ommission": 5
  }
}<|im_end|>



Sentence 10

Reference sentence She woke up early to catch the train

Target sentence She waked up early to catch the train

 {
  "errorSpan": ["mat sat on a cat"],
  "errorCategory": ["Unnatural"],
  "severity": ["Major"]
}<|im_end|>



Sentence 11

Reference sentence He wanted to go home but it was already too late

Target sentence He wanted to go home, but it was already too late.

 {
  "error_span": ["mat sat on a cat"],
  "error_category": "mistranslation",
  "severity": "minor"
}<|im_end|>



Sentence 12

Reference sentence The Statue of Liberty stands proudly in New York Harbor

Target sentence The Statue of Swatantrata stands proudly in New York Harbor

 {
  

## Gemma:2b

In [124]:
for i in range(0,len(df)):
    print('Sentence '+str(i)+'\n')
    print('Reference sentence '+df['Ref'][i]+'\n')
    print('Target sentence '+df['Tar'][i]+'\n')
    print(retres(ollama.Ollama(temperature=0,model='gemma:2b'),fewshotprompt,df['Ref'][i],df['Tar'][i],df['error_span'][i],df['severity'][i],df['error_cat'][i]))
    print('\n\n')

Sentence 0

Reference sentence A cat sat on a mat.

Target sentence A mat sat on a cat.

{
  "error_span": "mat sat on a cat",
  "error_category": "Unnatural",
  "severity": "Major"
}

{
  "error_span": "chair",
  "error_category": "Unnatural",
  "severity": "Major"
}

{
  "error_span": "one of your legs",
  "error_category": "Mistranslation, Addition",
  "severity": "Major"
}

{
  "error_span": "create your home",
  "error_category": "Mistranslation",
  "severity": "Major"
}

{
  "error_span": "it is raining, she went for a walk",
  "error_category": "Grammatical error",
  "severity": "Major"
}

{
  "error_span": "they has been studying French for three years",
  "error_category": "Grammatical error",
  "severity": "Major"
}

{
  "error_span": "She will arrive at the airport at noon tomorrow",
  "error_category": "Ommision",
  "severity": "Minor"
}

{
  "error_span": "We have completed the project ahead of schedule",
  "error_category": "Addition, Ommision",
  "severity": "Minor"
}

{

In [125]:
from datetime import datetime
print(datetime.now().strftime("%H:%M:%S"))

00:52:39
